## Testing

In [1]:
import fitz  # PyMuPDF

# Open the original PDF
doc = fitz.open("trojan_ok.pdf")

# Create a new PDF to hold the selected pages
new_doc = fitz.open()

# Page numbers are 0-based, so 21st page is index 20
new_doc.insert_pdf(doc, from_page=0, to_page=10)  # inclusive range

# Save the result
new_doc.save("trojan_1.pdf")


<class 'ModuleNotFoundError'>: No module named 'fitz'

In [2]:
from openai import AzureOpenAI
# Load environment variables
endpoint = "https://gpt-omnia-sweden-1.openai.azure.com/"
api_key = "b767c605d00e470480e0105aad3ce76e"
deployment = "gpt-4o-mini-omnia-1"


# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",  # Or the latest API version you're using
)

# # Example: Creating a completion request
completion = client.chat.completions.create(
     model=deployment,  # This should be your deployment name
     messages=[{"role": "user", "content": "Tell me a joke"}]
)
assistant_message = completion.choices[0].message.content.strip()
assistant_message

<class 'ModuleNotFoundError'>: No module named 'openai'

In [2]:
prompt = """
You are a creative learning assistant. Your task is to generate an engaging, gamified storytelling learning experience using only the information from the provided PDF. Do not use any external knowledge.

Create a short interactive story (around 3–5 scenes) where students go through a journey and must answer open-ended questions at each step to progress.

Each scene should include:
- A short, engaging narrative.
- One question based ONLY on the content from the PDF.
- No answer options — the student will type their own answer.
- A clear prompt to the user like: "What is your answer?" and then wait for input.

After the student replies with their answer, the assistant should:
- Check the answer against the PDF.
- Say whether it's correct or incorrect.
- Briefly explain the correct answer using only PDF content.

Story Requirements:
- Use simple, engaging language suitable for students.
- The theme of the story can be fun (e.g. fantasy, mystery, sci-fi), but all educational content must come strictly from the PDF.
- Avoid using any information not found in the PDF.

Format:
Scene #: [Scene Title]
Narrative: [Story content]
Question: [Ask a clear, open-ended question based on the PDF]
Prompt: "What is your answer?"

(Wait for user input, then proceed to check and explain.)
"""

In [2]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text

# Load PDF content
pdf_text = extract_text_from_pdf("trojan_1.pdf")

# Truncate or chunk the PDF content if it's too large
max_chars = 4000  # Adjust based on model context length and prompt size
pdf_text = pdf_text[:max_chars]

In [3]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-4o-mini")
tokens = tokenizer.encode(pdf_text)
num_tokens = len(tokens)
num_tokens

973

In [5]:
# Combine PDF content with your prompt
combined_prompt = f"""
Use only the following content from the PDF for your responses:

\"\"\"
{pdf_text}
\"\"\"

Now, based on the above, follow the instructions below.

{prompt}
"""

# Call Azure OpenAI ChatCompletion
response = client.chat.completions.create(
    model=deployment,  # Your deployment name
    messages=[
        {
            "role": "system",
            "content": (
                "You are a creative and educational assistant. Your task is to design an interactive, gamified storytelling experience for students, "
                "based strictly on the content of a provided PDF. Do not use any external knowledge. In each scene, guide the student through a story, "
                "ask open-ended questions based only on the PDF, and wait for their input before evaluating the response."
            )
        },
        {
            "role": "user",
            "content": combined_prompt
        }
    ],
    max_tokens=1000,  # You can increase this depending on story length
    temperature=0.7,  # Slightly more creative
    top_p=1,
    n=1,
    stop=None,
)

# Get and print the output
generated_text = response.choices[0].message.content.strip()
print("Generated:")
print(generated_text)

Generated:
,**Scene 1: The Spark of Conflict**  
,Narrative: In ancient Greece, a beautiful woman named Helen was taken from her husband, Menelaus, the king of Sparta. This act sparked a legendary war that would be told for generations. The Achaeans, known as the Greeks, gathered their forces to march against the city of Troy, where Helen had been taken. As the warriors prepared for battle, a sense of fate hung in the air. What could have driven Paris of Troy to take such a bold step?  
,Question: Why did the Achaeans wage war against the city of Troy?  
,Prompt: "What is your answer?"  
,
,(Wait for user input, then proceed to check and explain.)


In [36]:
print("\n" + "="*80)
print("🎮  Gamified Learning Journey")
print("="*80 + "\n")

print(generated_text.strip())

print("\n" + "="*80)
print("📝  Your Turn: Please answer the question above to continue the journey!")
print("="*80 + "\n")



,================================================================================
,🎮  Gamified Learning Journey
,================================================================================
,
,**Scene 1: The Call to Adventure**  
,Narrative: In a distant land where myths come alive, you find yourself in ancient Greece, just before a great conflict known as the Trojan War. The air is filled with tension as Achaeans prepare to march against the city of Troy, all because Paris of Troy has taken Helen, the wife of Menelaus, the king of Sparta. You stand at the edge of the battlefield, feeling the weight of history around you. Will you join the Achaeans in their quest?  
,
,Question: Why did the Achaeans go to war against Troy?  
,Prompt: "What is your answer?"  
,
,(Wait for user input.)
,
,================================================================================
,📝  Your Turn: Please answer the question above to continue the journey!
,==========================================

In [40]:
def predict(input, history=[]):
    # Define instructions and knowledge base
    instruction = 'Instruction: given a dialog context, you need to respond empathically'
    knowledge_base = {
        "what is google colab": ("Google Colab, or 'Colaboratory,' is a free cloud service provided by Google that allows you to write and execute Python code in a web-based environment. "
                                 "It's particularly useful for data science, machine learning, and AI projects. Key features include no setup required, free access to GPUs and TPUs, "
                                 "interactive notebooks, and integration with Google Drive.")
    }

    # Flatten the history and append the new input
    s = list(sum(history, ()))
    s.append(input)
    dialog = ' EOS '.join(s)
    query = f"{instruction} [CONTEXT] {dialog}"

    # Check if the input matches any knowledge base entries
    input_lower = input.lower()
    if input_lower in knowledge_base:
        response = knowledge_base[input_lower]
    else:
        # Generate a response using the model
        new_user_input_ids = tokenizer.encode(f"{query}", return_tensors='pt')
        output = model.generate(new_user_input_ids, min_length=8, max_length=64, top_p=0.9, do_sample=True).tolist()
        response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Update the history
    history.append((input, response))

    return history, history

In [42]:
import openai

response = client.chat.completions.create(
    model=deployment,  # Your deployment name
    messages=[
        {
            "role": "system",
            "content": (
                "You are a creative and educational assistant. Your task is to design an interactive, gamified storytelling experience for students, "
                "based strictly on the content of a provided PDF. Do not use any external knowledge. In each scene, guide the student through a story, "
                "ask open-ended questions based only on the PDF, and wait for their input before evaluating the response."
            )
        },
        {
            "role": "user",
            "content": combined_prompt
        }
    ],
    max_tokens=1000,  # You can increase this depending on story length
    temperature=0.7,  # Slightly more creative
    top_p=1,
    n=1,
    stop=None,
)

# Print the response
response.choices[0].message.content.strip()
print(response.choices[0].message.content.strip())

**Scene 1: The Call to Adventure**  
,Narrative: You find yourself in ancient Greece, where the sun shines brightly over the bustling city of Sparta. Suddenly, a messenger rushes in, breathless and alarmed. "The beautiful Helen has been taken by Paris of Troy!" he shouts. This act has angered King Menelaus, and he gathers the greatest warriors to sail to Troy and reclaim her. You feel a stirring in your heart — could you join them on this legendary quest?  
,Question: Why did the Achaeans go to war against the city of Troy?  
,Prompt: "What is your answer?"  
,
,---
,
,(Wait for user input, then proceed to check and explain.)


## Main Code

In [1]:
import json
import gradio as gr
import fitz  # PyMuPDF
from openai import AzureOpenAI
import re
import copy
# Load environment variables
endpoint = "https://gpt-omnia-sweden-1.openai.azure.com/"
api_key = "ììì"
deployment = "gpt-4o-mini-omnia-1"


# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",  # Or the latest API version you're using
)

In [2]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text

# Load PDF content
pdf_text = extract_text_from_pdf("trojan_1.pdf")

# Truncate or chunk the PDF content if it's too large
max_chars = 4000  # Adjust based on model context length and prompt size
pdf_text = pdf_text[:max_chars]

In [ ]:
# Main gamified assistant history
chat_history = [
]

# Secondary helper assistant history
helper_chat_history = [
]

answer_question_list = [
]

# Main story handler
def chat_with_story(user_input):
    chat_history.append({"role": "user", "content": user_input})

    
    assistant_question = chat_history[-2]["content"]
    user_answer = chat_history[-1]["content"]
        
    evaluator_prompt = f"""
    You are the **evaluator agent** in a multi-agent educational system.
    
    Your task is to assess the correctness of the student's answer using **only the content provided in the PDF**. You must not rely on any external knowledge.
    
    Below is the relevant information:
    
    📚 PDF Content:
    \"\"\"
    {pdf_text}
    \"\"\"
    
    🤖 Assistant's Question:
    {assistant_question}
    
    👤 Student's Answer:
    {user_answer}
    
    ---
    
    🎓 Instructions:
    
    1. Compare the student's answer to what is stated or implied in the PDF.
    2. If the answer aligns with the PDF, it is correct. If it is incomplete, incorrect, or fabricated, mark it as incorrect.
    3. Extract the most accurate answer from the PDF (i.e., the correct answer).
    4. Assign a **score** between 0 and 1:
       - `1` for completely correct
       - `0` for incorrect
       - Any value in between (e.g., 0.5) for partially correct answers
    
    5. Output your evaluation in this strict JSON format:
    
    ```json
    {{
      "AssistantQuestion": "<Restate the original assistant question>",
      "UserAnswer": "<Restate the user's answer>",
      "CorrectAnswer": "<The correct answer based strictly on the PDF>",
      "Explanation": "<Explain why the answer is correct or incorrect>",
      "Judgment": "Correct" or "Incorrect",
      "Score": <float between 0 and 1>
    }}
    
    ⚠️ Notes:
    
    - Use short quotes from the PDF if helpful.
    
    - Do not create new story content or repeat the narrative.
    
    - Only output the JSON — no extra commentary or formatting.
    """
    chat_with_evaluation = copy.deepcopy(chat_history)
    evaulation_judg = "tmp"
    
    if len(chat_history) > 3:
        response = client.chat.completions.create(
            model=deployment,
            messages=[{"role": "user", "content": evaluator_prompt}],
            max_tokens=1000,
            temperature=0.1,
            top_p=1
        )
            
        evaluation_answer = response.choices[0].message.content.strip()
        json_pattern = re.compile(r'(\{.*\})', re.DOTALL)
        match = json_pattern.search(evaluation_answer)
        evaluation_str = match.group(1)
        json_evaulation = json.loads(evaluation_str)
        answer_question_list.append(json_evaulation)
        chat_with_evaluation.append({"role": "assistant", "content": f"[Evaluator Agent] {evaluation_str}"})
    
        evaulation_judg = json_evaulation["Judgment"]

    response = client.chat.completions.create(
        model=deployment,
        messages=chat_with_evaluation,
        max_tokens=1000,
        temperature=0.7,
        top_p=1
    )
    reply = response.choices[0].message.content.strip()
    chat_history.append({"role": "assistant", "content": f"[Assistant Agent] {reply}"})
    
    if(evaulation_judg == "Incorrect"):
        helper_chat_history.append({"role": "assistant", "content": f"[Evaluator Agent] {evaluation_str}"})
        helper_response = chat_with_helper(user_input, include_user=True)
        return "", chat_history[1:], helper_response
    return "", chat_history[1:], helper_chat_history[1:]

# Helper assistant handler
def chat_with_helper(user_input, include_user=True):
    if include_user:
        helper_chat_history.append({"role": "user", "content": f"[User Answer] {user_input}"})

    response = client.chat.completions.create(
        model=deployment,
        messages=helper_chat_history,
        max_tokens=500,
        temperature=0.3,
        top_p=1
    )
    reply = response.choices[0].message.content.strip()
    helper_chat_history.append({"role": "assistant", "content": f"[Helper Agent] {reply}"})
    #return "", helper_chat_history[1:]
    return "", helper_chat_history[1:]

# Start story on load
def start_story():

    helper_system_prompt = """
    You are a helpful learning assistant designed to support students in reasoning their way toward correct answers without simply giving the solution.
    
    You will receive structured evaluation data in this format:
    
    {
      "AssistantQuestion": "<Restate the original assistant question>",
      "UserAnswer": "<Restate the user's answer>",
      "CorrectAnswer": "<The correct answer based strictly on the PDF>",
      "Explanation": "<Explain why the answer is correct or incorrect>",
      "Judgment": "Correct" or "Incorrect",
      "Score": <float between 0 and 1>
    }
    
    Your role is to:
    1. Understand the assistant's original question and the concept being tested.
    2. Reflect on the student’s answer and the evaluator's explanation.
    3. **Help the student improve their understanding** of the topic through:
       - Socratic questioning 🤔
       - Gentle clues and scaffolding 💡
       - Encouraging deeper thinking 🧠
    
    **Do not immediately reveal the correct answer.** Instead:
    - If the answer is partially correct (Score between 0.3–0.7), encourage refinement.
    - If incorrect, prompt the student to rethink the question using clues from the concept.
    - If correct, briefly reinforce the reasoning and encourage progress to the next scene.
    
    Tone: supportive, motivational, and adaptive. Use second-person language ("you") and keep the student engaged without frustration. Your goal is to help them **learn through discovery**, not just receive answers.
    
    Never output the correct answer directly unless explicitly instructed.
    """
    
    initial_prompt = f"""
    You are an intelligent educational agent. Your task is to analyze the provided PDF content (in the variable `pdf_text`) and identify the academic or cognitive skills that can be directly assessed based on its content.
    
    Instructions:
    
    1. Carefully analyze the entire content in `pdf_text`.
    2. Identify only the skills that are explicitly testable using the actual content. Do not guess or include generic skills unless they are clearly supported by the text.
    3. Each skill must be categorized using **Bloom's Taxonomy**, using one of the following levels:
       - Remember
       - Understand
       - Apply
       - Analyze
       - Evaluate
       - Create
    4. Choose the most appropriate Bloom’s level for each skill based on how a student would be challenged using the PDF content.
    
    ---
    
    Output Format (strict JSON):
    
    ```json
    {{
      "Skills": [
        "Historical fact recall – Bloom: Remember",
        "Cause and effect reasoning – Bloom: Analyze",
        "Source evaluation – Bloom: Evaluate"
      ]
    }}
    
    
    Use only the content in {pdf_text}. Do not use any outside knowledge. Make sure the output is in valid JSON and includes only those three fields.
    """
    response = client.chat.completions.create(
        model=deployment,
        messages=[{"role": "user", "content": initial_prompt}],
        max_tokens=500,
        temperature=0.3,
        top_p=1
    )
    generated_text = response.choices[0].message.content.strip()
    json_pattern = re.compile(r'(\{.*\})', re.DOTALL)
    match = json_pattern.search(generated_text)
    json_str = match.group(1)
    json_init = json.loads(json_str)

    init_skills = json_init["Skills"]

    combined_prompt = f"""
    You are in the role of the **assistant** in a multi-agent educational system.
    
    Your task is to generate a gamified, interactive storytelling experience based strictly on the content of the provided PDF. Another agent, called the **evaluator**, will be responsible for judging the correctness of the student's answers. You must NOT evaluate answers yourself.
    
    Here is the source content you must base your story on:
    
    \"\"\"
    {pdf_text}
    \"\"\"
    
    The student should be tested on the following academic skills (mapped to Bloom's Taxonomy):
    
    {init_skills}
    
    Now, using only the information from the PDF, create a highly engaging learning journey.
    
    ### Instructions for the Assistant:
    
    - You are the storyteller and guide.
    - Write in **second-person perspective ("you")**, making the student the main character.
    - Make the story immersive and gamified — e.g., a quest 🧭, survival ⚔️, time-travel mystery ⏳, or sci-fi journey 🚀.
    - Use **emojis** to enhance emotion, atmosphere, and decision points: 📜🧠🧩⚖️🔥
    
    ### Each scene must contain:
    - A vivid, engaging narrative grounded strictly in the PDF.
    - One open-ended question tied to the skills above.
    - A prompt: "📝 What is your answer?"
    
    After the user responds:
    - The **evaluator agent** will assess whether the answer is correct, using only the PDF content.
    - You must then continue the story based on the evaluator's judgment:
    
        - ✅ If the evaluator says the answer is correct:
            - Congratulate the student with excitement and progression in the story.
            - Use celebratory emojis (🎉🚀🟢) and move the story forward.
    
        - ❌ If the evaluator says the answer is incorrect:
            - Guide the student with encouragement.
            - Clarify the correct concept using only what the evaluator said.
            - Help them recover and continue the story with support (💡📘🔄).
    
    You must never guess, invent, or judge correctness yourself. Always wait for the evaluator's decision before proceeding.
    
    Your goal is to make the learning journey feel immersive, personal, and adaptive — just like a dynamic learning game 🎯.
    """
    if len(chat_history) == 0 and len(helper_chat_history) == 0:
        helper_chat_history.append({"role": "system", "content": helper_system_prompt})
    
        chat_history.append({"role": "system", "content": combined_prompt})
    
        intro_message = "Welcome to the story adventure! Let's begin your journey."
        help_intro = "Hey, we’re your colleagues — let’s talk through the topic you’re stuck on together."
        
        chat_history.append({"role": "assistant", "content": f"[Assistant Agent] {intro_message}"})
        helper_chat_history.append({"role": "assistant", "content": f"[Helper Agent] {help_intro}"})

    return chat_history[1:], helper_chat_history[1:]

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Main Learning Chat (Gamified Story)")

    with gr.Row():
        chatbot = gr.Chatbot(
            label="Gamified Story Assistant",
            type="messages",
            height=600,
            avatar_images=["user.jpg", "assistant.jpg"]  # 👤 🤖
        )

        helpbot = gr.Chatbot(
            label="Help Assistant",
            type="messages",
            height=350,
            avatar_images=["user.jpg", "wizard.jpg"]  # 👤 🧙
        )

    with gr.Row():
        msg = gr.Textbox(label="Your Answer", placeholder="Type your story answer here...")
        helper_input = gr.Textbox(label="Ask for Help", placeholder="How do I progress?")

    msg.submit(chat_with_story, [msg], [msg, chatbot, helpbot])
    helper_input.submit(chat_with_helper, [helper_input], [helper_input, helpbot])

    demo.load(start_story, outputs=[chatbot, helpbot])


In [ ]:
demo.launch(debug=True, share=True)

* Running on local URL:  http://127.0.0.1:7860
,* Running on public URL: https://eacd0b651cadb8eed7.gradio.live
,
,This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
answer_question_list

[{'AssistantQuestion': 'What event triggered the Trojan War between the Achaeans and the city of Troy?',
  'UserAnswer': 'paris kidnapped wife of sparta',
  'CorrectAnswer': 'The war was waged by the Achaeans (Greeks) against the city of Troy after Paris of Troy took Helen from her husband Menelaus, king of Sparta.',
  'Explanation': "The student's answer captures the essence of the event that triggered the Trojan War, specifically mentioning Paris and the kidnapping of Helen. However, it lacks clarity and completeness, as it does not specify that Helen was taken from her husband Menelaus, which is a crucial part of the context.",
  'Judgment': 'Incorrect',
  'Score': 0.5}]

In [ ]:
chat_history